## Census Block API Call + Merge to Block Shapefile Code

#### I created this Python file to help my research team use the census's API to call data and merge it to a shapefile.

In [ ]:
# Import the Python packages that we need to request the data and process it.
import requests as rq
import pandas as pd

In [ ]:
# Get a census api key from: https://api.census.gov/data/key_signup.html
with open('census-api-key.txt') as key:
    api_key=key.read().strip()
    
# For the list of available tables, go to: https://www.census.gov/data/developers/data-sets.html 
dsource='acs'
dname='acs5'

In [ ]:
# This section of code is adapted from online instructions available from Francis Donnelly available at the following URL: https://atcoordinates.info/2019/09/24/examples-of-using-the-census-bureaus-api-with-python/
# Call your API Key text file to read your key"
with open('census-api-key.txt') as key:
    api_key=key.read().strip()
    
# For the list of available tables, go to: https://www.census.gov/data/developers/data-sets.html 
dsource='acs'
dname='acs5'

col_pop = 'GEO_ID,B01001_001E'
cols_race = 'GEO_ID,B03002_001E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_013E,B03002_014E,B03002_015E,B03002_016E,B03002_017E,B03002_018E,B03002_019E'

In [ ]:
# * will retrieve ALL tracts.  If you want only a specific tract, enter the 6 digit number in quotes here
blkgrp='*'

# This specifies the state.
state='36'

# This specifies the counties.  Note, there is no space between the numbers.
county='005,047,061,081,085'

In [ ]:
#Call the data using the API
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'
data_url = f'{base_url}?get={cols_pop}&for=block%20group:{blkgrp}&in=state:{state}&in=county:{county}&key={api_key}'
response=rq.get(data_url)

In [ ]:
#Convert the data to a data frame
data=response.json()
print(data)
df=pd.DataFrame(data[1:], columns=data[0])

path = 'path/to/your/csv.csv'
df.to_csv(path, index=False)

In [ ]:
# This section of the code is to merge the blocks we called from the API to the shapefile.
censuscsv = 'path/to/the/census/csv.csv'
censusshapefile = 'path/to/the/censusblocks/shapefile.shp'

shp_df = gpd.read_file(censusshapefile)
csv_df = pd.read_csv(censuscsv, dtype={'state': str, 'county': str, 'tract': str})

csv_df['GeoID'] = csv_df['state'] + csv_df['county'] + csv_df['tract'] +csv_df['block']

shp_df = shp_df.rename(columns={'GEOID': 'GeoID2'})
csv_df = csv_df.rename(columns={'GeoID': 'GeoID2'})

merged_df = shp_df.merge(csv_df, on='GeoID2')
newpath = 'path/where/you/want/your/shapefile/to/output.shp'
merged_df.to_file(newpath, index=False)


In [ ]:
# Or if you need to do a calculation, sub out your columns here: 
merged_df['per_transit'] = ((merged_df['B08301_010E'] / merged_df['B08301_001E']) * 100)
merged_df.to_file(newpath, index=False)